In [1]:
# setting device on GPU if available, else CPU
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')


Using device: cuda

NVIDIA GeForce RTX 2080 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import os
import random
from typing import Callable, Dict, List, Optional, Tuple

import numpy as np
import torch
from torch.utils import data
from tqdm import tqdm



In [4]:
from configs.config import get_cfg_defaults
from core.datasets.dataset_loading_utils import load_dataset
from core.datasets.vq_dataset import DATALoader
from utils.vis_utils import plot_3d_global
from core.models.conformer_vqvae import ConformerVQMotionModel, Encoder
from torch.utils import data
from core.datasets.vq_dataset import DATALoader, MotionCollator
from einops import pack, rearrange, reduce, repeat, unpack

def pack_one(t, pattern):
    return pack([t], pattern)


In [5]:
from utils.vis_utils.render_final import Renderer
renderer = Renderer(device)

In [6]:
path = "/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/checkpoints/conformer_768_1024_affine_varlen/conformer_768_1024_affine_varlen.yaml"
cfg = get_cfg_defaults()
print("loading config from:", path)
cfg.merge_from_file(path)
cfg.freeze()

loading config from: /srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/checkpoints/conformer_768_1024_affine_varlen/conformer_768_1024_affine_varlen.yaml


In [ ]:
from core.models.conformer_vqvae import ConformerVQMotionModel, Encoder
convvq = ConformerVQMotionModel(cfg.vqvae).to(device).eval()
convvq.load("/srv/hays-lab/scratch/sanisetty3/music_motion/TGM3D/checkpoints/conformer_512_1024_affine/vqvae_motion.pt")

In [7]:
from core.datasets.vq_dataset import VQMotionDataset


In [122]:
train_ds = VQMotionDataset("t2m" , "/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotionSMPL" , window_size = -1, split = "test")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 6221/6221 [00:04<00:00, 1471.00it/s]

Total number of motions 6221


In [123]:
train_dl = DATALoader(
            train_ds,
            batch_size=1,
            shuffle=False,
            collate_fn=None,
        )

In [124]:
for batch in train_dl:
    break

In [127]:
dest = "/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotionSMPLIndices/HumanML3D_SMPL/joint_indices"

In [128]:
for i, batch in enumerate(tqdm(train_dl)):
    # if i < 12300:
    #     continue
    
    gt_motion = batch["motion"].to(device)
    if gt_motion.shape[1] > 1000:
        ind = []
        for m in range(0, gt_motion.shape[1], 1000):
            indics = convvq.encode(gt_motion[:, m:m+1000])
            ind.append(indics[0])
        indices = torch.cat(ind)[None]
    else:
        indices = convvq.encode(gt_motion)
    np.save(os.path.join(dest , batch["names"][0]+".npy") , indices.detach().cpu().numpy())
    del indices
    del gt_motion
    torch.cuda.empty_cache()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 6221/6221 [04:07<00:00, 25.11it/s]


In [28]:
# from glob import glob
# glob("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotionSMPLIndices/HumanML3D_SMPL/joint_indices/*")

In [107]:
og = "/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotionSMPL/Choreomaster_SMPL/new_joint_vecs/00_1093.npy"

In [108]:
src = "/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotionSMPLIndices/Choreomaster_SMPL/joint_indices/00_1093.npy"

In [109]:
ind = torch.Tensor(np.load(src))
ogg = torch.Tensor(np.load(og))

In [110]:
ind.shape

torch.Size([1, 866])

In [111]:
quantized, decoded_motion_features = convvq.decode(ind.long())


In [112]:
decoded_motion_features.shape

torch.Size([1, 3464, 271])

In [113]:
ogg.shape

torch.Size([3464, 271])

In [114]:
renderer.render(
    motion_vec=train_ds.inv_transform(decoded_motion_features)[0,:1000,:135],
    outdir="./renders/",
    step=0,
    name=f"00_000007_pred",
)

In [115]:
renderer.render(
    motion_vec=ogg[:1000,:135],
    outdir="./renders/",
    step=0,
    name=f"00_000007_og",
)

In [136]:
hml = "/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotion/HumanML3D/Std.npy"
hml2 = "/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotion/AIST/Std.npy"
hml3 = "/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotion/Choreomaster/Std.npy"

In [137]:
np.save("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotion/Std.npy" , np.mean([np.load(hml) + np.load(hml2) + np.load(hml3)] , 0))

In [138]:
from glob import glob

In [139]:
pths = sorted(glob("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotion/HumanML3D/new_joint_vecs/*"))

In [145]:
pths[25000]

'/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotion/HumanML3D/new_joint_vecs/M008676.npy'

In [144]:
int(pths[25000].split("/")[-1].split(".")[0][-6:])

8676

In [156]:
add = []
for p in tqdm(pths):
    nm = int(p.split("/")[-1].split(".")[0][-6:])
    if nm > 14616:
        add.append(p.split("/")[-1].split(".")[0])
        

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 32648/32648 [00:00<00:00, 964613.99it/s]


In [158]:
with open(r'/srv/hays-lab/scratch/sanisetty3/music_motion/HumanMotion/HumanML3D/train.txt', 'a') as fp:
    for item in add:
        fp.write("%s\n" % item)
    print('Done')

Done


In [157]:
len(add)

3418

In [153]:
add = []
for p in tqdm(pths):
    n = p.split("/")[-1].split(".")[0]
    add.append(n)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 32648/32648 [00:00<00:00, 1130149.03it/s]


Done


## VQVARLEN

In [20]:
#from utils.vis_utils.write_obj import main
# main(model_folder="/srv/hays-lab/scratch/sanisetty3/music_motion/motion_vqvae/body_models/smplh/SMPLH_NEUTRAL.npz", motion_file = "/srv/hays-lab/scratch/sanisetty3/music_motion/HumanML3D/amass_data/ACCAD/Male1Running_c3d/Run C24 - quick side step left_poses.npz" , output_folder = "./renders/" )

In [14]:
from glob import glob

In [25]:
hml = sorted(glob("/srv/hays-lab/scratch/sanisetty3/music_motion/HumanML3D/HumanML3D/new_joint_vecs/*"))

In [26]:
(hml[17000].split("/")[-1].split(".")[0][-6:])

'000676'

In [28]:
lens = []
for i in tqdm(hml):
    name = int(i.split("/")[-1].split(".")[0][-6:])
    if name > 14616:
        continue

    lens.append(np.load(i).shape[0])


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 32648/32648 [00:13<00:00, 2382.97it/s]


In [72]:
from core.models.conformer import ConformerBlock

In [66]:
enc = Encoder()

In [57]:
for batch in valid_dl:
    break

In [58]:
gt_motion = batch["motion"].to(device)
mask = batch.get("motion_mask", None).to(device)

In [66]:
mask[1].shape

torch.Size([105])

In [121]:

for l in enc.blocks:
    if isinstance(l, ConformerBlock):
        print(x.shape)
        x = l(x,mask)
        mask = torch.nn.functional.max_pool1d(mask.float() , 3 ,stride = 2, padding = 1).bool()
    else:
        x = l(x)

torch.Size([4, 80, 768])
torch.Size([4, 40, 768])


In [62]:
mask1 = torch.nn.functional.max_pool1d(mask.float() , 3 ,stride = 2, padding = 1).bool()
mask2 = torch.nn.functional.max_pool1d(mask1.float() , 3 ,stride = 2, padding = 1).bool()

In [90]:
mask1.shape

torch.Size([4, 53])

In [63]:
mask2.shape

torch.Size([4, 27])

In [96]:
mask3 = torch.nn.functional.upsample(mask2[None].float() , scale_factor=2, mode="nearest")[0, :,:-1]
# mask3 = torch.nn.functional.max_pool1d(mask3 , 3 ,stride = 1, padding = 1)
mask4 = torch.nn.functional.upsample(mask3[None].float() , scale_factor=2, mode="nearest")[0, :,:-1]
# mask4 = torch.nn.functional.max_pool1d(mask4 , 3 ,stride = 1, padding = 1)


In [97]:
mask3[1]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')

In [98]:
mask1[1].float()

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       device='cuda:0')

In [104]:
mask4[1].long()

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [105]:
mask[1]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')

In [106]:
int(np.log2(4))

2

In [136]:
import pickle

In [170]:
with open("/srv/hays-lab/scratch/sanisetty3/music_motion/MOYO/mosh2/train/220923_yogi_body_hands_03596_Boat_Pose_or_Paripurna_Navasana_-a_stageii.pkl" , 'rb') as f:
    data = pickle.load(f)

In [171]:
# data["markers_latent"]

In [172]:
data_ = {

    "poses":data["fullpose"],
    "trans" : data["trans"],
"betas" : data["betas"],
"mocap_framerate" : 30,
"gender":"female",

}

In [177]:
np.savez("/srv/hays-lab/scratch/sanisetty3/music_motion/MOYO/test1.npz" , **data_)

In [178]:
with np.load("/srv/hays-lab/scratch/sanisetty3/music_motion/MOYO/test1.npz") as data:
    # Check for valid AMASS file
    print(data)
    if ("trans" not in data) or ("gender" not in data) or (("mocap_frame_rate" not in data) and ("mocap_framerate" not in data)) or ("betas" not in data) or ("poses" not in data):
        print("Invalid AMASS animation data file")


In [174]:
data = np.load("/srv/hays-lab/scratch/sanisetty3/music_motion/MOYO/test1.npz") 